# Fase 1 – Creación de Archivos
Crear carpetas data, database, reports, ci, git y scripts y estructura base del proyecto usando os y pathlib en tiempo de ejecución


In [1]:
from pathlib import Path

# Crear estructura base
base_path = Path("SaludDigitalB_E1")

folders = [
    "data", "database", "reports", "ci", "git", "scripts"
]

for folder in folders:
    (base_path / folder).mkdir(parents=True, exist_ok=True)

print("Estructura creada correctamente ✅")


Estructura creada correctamente ✅
